In [5]:
import re
import os
from pathlib import Path
import shutil

In [6]:
cv_path = str(Path(os.getcwd()).parents[0])
cv_file = cv_path + '/cv.md'
# cv_path = str(Path(os.getcwd()).parents[0]) # cv_path = 'D:/OneDrive - University of California, Davis/43_github/EunseongBae.github.io/cv.md'
cv_raw = open(cv_file, 'r')
cv_lines = cv_raw.readlines()
cv_raw.close()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 107: character maps to <undefined>

In [15]:
cv_crit1 = {'author','position','affiliation1','affiliation2','date'}
cv_crit2 = {'email','homepage'}

count = 0
cv_sep = 0
cv_info = {}

# cv_info3 = {'education', 'work experience', 'teaching experience', 'publications'}

for line in cv_lines:
    count += 1
    if line == '---\n' :
        cv_sep += 1
        continue
    
    if (cv_sep == 1) & (':' in line) :
        a = line.split(':')[0]
        if a in cv_crit1:
            cv_info[a] = line.split(':')[1].strip()
        elif a in cv_crit2:
            cv_info[a] = cv_lines[count+1].split(':')[1].strip()
    
    if cv_sep == 2:
        break        

res1 = open('cv_latex1.txt', 'w')

res1.write( '\\begin{center} \n{\Huge\n'
    + cv_info['author'] 
    + '} \\\[1em] \n'
    + cv_info['position'] +', '
    + cv_info['affiliation1'] + ' \\\ \n'
    + cv_info['affiliation2'] + ' \\\ \n'
    + '\href{mailto:'+ cv_info['email']+'}{' + cv_info['email'] +'} \quad \n'
    + '\href{https://'+ cv_info['homepage']+'}{' + cv_info['homepage'] +'} \quad \n'
    + 'Updated: '+ cv_info['date']
    + '\n\\end{center}' + '\n \\vspace{1em} \n'
    + '%'*20)

res1.close()

In [16]:
def check_space(string):
    count = 0
    for i in range(0, len(string)):
        if string[i] == ' ' :
            count += 1
        else :
            return(count)

cv_info2 = {}
count = 0

for line in cv_lines[count:] :
    if len(line.strip()) > 0  :
        if line[:2] == "##" :
            temp_name = line[2:].strip()
            cv_info2[temp_name] = []
            count += 1
            continue
        if count !=0 :
            x = re.split('<[^>]+>', line)
            if len(x) > 1 :
                line = x[0]+'\href{'+x[1]+'}{'+x[1]+'}'
            line = line.replace('\n','')
            line = line.replace('&','\&')
            line = re.sub('<[^>]+>', '', line)
            cv_info2[temp_name].append(line)

res2 = open('cv_latex2.txt', 'w')
res2.write('\n')

for tag in cv_info2 :
    # if tag in {'Education','Work Experience','Publications'} :
    #     continue

    res2.write('{\Large '+tag+'}'+'\n\n')
    res2.write("\\begin{itemize} \n ")

    num_space_old = 0
    num_space = 0
    for x in cv_info2[tag] :
        num_space = check_space(x) // 2
        y = x[(num_space+2):]
        if num_space > num_space_old :
            res2.write("\t"*(num_space) + "\\begin{itemize}[leftmargin=1.5em] \n")
            res2.write("\t"*(num_space+1) +"\item " + y + "\n")
        elif num_space == num_space_old :
            res2.write("\t"*(num_space+1) +"\item " + y + "\n")
        else :
            res2.write("\t"*(num_space+1) + "\\end{itemize} \n")
            res2.write("\t"*(num_space+1) +"\item "+ y + "\n")
        num_space_old = num_space
    res2.write('\\end{itemize} \n' * (num_space+1))
    res2.write('\n')

res2.write('\\end{document}')
res2.close()

In [17]:
filenames = ['cv_latex0.txt', 'cv_latex1.txt', 'cv_latex2.txt']
with open('cv_latex.tex', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            outfile.write(infile.read())
outfile.close()

In [18]:
cmd = 'pdflatex -synctex=1 -interaction=nonstopmode --shell-escape "cv_latex".tex'
os.system(cmd)

0

In [19]:
src_path = cv_path + '\cv_pdf\cv_latex.pdf'
dst_path = cv_path + '\downloads\CV_' + cv_info['author'] + '.pdf'
shutil.copy(src_path, dst_path)

'd:\\OneDrive - University of California, Davis\\43_github\\EunseongBae.github.io\\downloads\\CV_Eunseong Bae.pdf'